# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 2 2022 10:20AM,243255,10,SONSB0001889,"Nextsource Biotechnology, LLC",Released
1,Aug 2 2022 10:19AM,243242,20,8347761,"ACI Healthcare USA, Inc.",Released
2,Aug 2 2022 10:12AM,243252,15,8347527,"Carwin Pharmaceutical Associates, LLC",Released
3,Aug 2 2022 10:08AM,243250,19,8351615,"Emersa Waterbox, LLC",Released
4,Aug 2 2022 10:06AM,243249,19,8351613,"Emersa Waterbox, LLC",Released
5,Aug 2 2022 10:05AM,243248,15,8347850,"Brookfield Pharmaceuticals, LLC",Released
6,Aug 2 2022 10:04AM,243174,15,8347590,"Brookfield Pharmaceuticals, LLC",Released
7,Aug 2 2022 10:04AM,243174,15,8347603,"Brookfield Pharmaceuticals, LLC",Released
8,Aug 2 2022 10:04AM,243174,15,8347604,"Brookfield Pharmaceuticals, LLC",Released
9,Aug 2 2022 10:04AM,243174,15,8347605,"Brookfield Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
37,243248,Released,1
38,243249,Released,1
39,243250,Released,1
40,243252,Released,1
41,243255,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243248,NaN,NaN,1.0
243249,NaN,NaN,1.0
243250,NaN,NaN,1.0
243252,NaN,NaN,1.0
243255,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243152,0.0,1.0,1.0
243170,8.0,37.0,0.0
243173,0.0,0.0,16.0
243174,0.0,0.0,37.0
243185,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243152,0,1,1
243170,8,37,0
243173,0,0,16
243174,0,0,37
243185,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243152,0,1,1
1,243170,8,37,0
2,243173,0,0,16
3,243174,0,0,37
4,243185,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243152,,1,1
1,243170,8,37,
2,243173,,,16
3,243174,,,37
4,243185,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 2 2022 10:20AM,243255,10,"Nextsource Biotechnology, LLC"
1,Aug 2 2022 10:19AM,243242,20,"ACI Healthcare USA, Inc."
2,Aug 2 2022 10:12AM,243252,15,"Carwin Pharmaceutical Associates, LLC"
3,Aug 2 2022 10:08AM,243250,19,"Emersa Waterbox, LLC"
4,Aug 2 2022 10:06AM,243249,19,"Emersa Waterbox, LLC"
5,Aug 2 2022 10:05AM,243248,15,"Brookfield Pharmaceuticals, LLC"
6,Aug 2 2022 10:04AM,243174,15,"Brookfield Pharmaceuticals, LLC"
43,Aug 2 2022 10:03AM,243173,15,"Alliance Pharma, Inc."
59,Aug 2 2022 10:02AM,243247,19,"Emersa Waterbox, LLC"
60,Aug 2 2022 10:00AM,243246,19,"Emersa Waterbox, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 2 2022 10:20AM,243255,10,"Nextsource Biotechnology, LLC",,,1
1,Aug 2 2022 10:19AM,243242,20,"ACI Healthcare USA, Inc.",,,1
2,Aug 2 2022 10:12AM,243252,15,"Carwin Pharmaceutical Associates, LLC",,,1
3,Aug 2 2022 10:08AM,243250,19,"Emersa Waterbox, LLC",,,1
4,Aug 2 2022 10:06AM,243249,19,"Emersa Waterbox, LLC",,,1
5,Aug 2 2022 10:05AM,243248,15,"Brookfield Pharmaceuticals, LLC",,,1
6,Aug 2 2022 10:04AM,243174,15,"Brookfield Pharmaceuticals, LLC",,,37
7,Aug 2 2022 10:03AM,243173,15,"Alliance Pharma, Inc.",,,16
8,Aug 2 2022 10:02AM,243247,19,"Emersa Waterbox, LLC",,,1
9,Aug 2 2022 10:00AM,243246,19,"Emersa Waterbox, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 2 2022 10:20AM,243255,10,"Nextsource Biotechnology, LLC",1,,
1,Aug 2 2022 10:19AM,243242,20,"ACI Healthcare USA, Inc.",1,,
2,Aug 2 2022 10:12AM,243252,15,"Carwin Pharmaceutical Associates, LLC",1,,
3,Aug 2 2022 10:08AM,243250,19,"Emersa Waterbox, LLC",1,,
4,Aug 2 2022 10:06AM,243249,19,"Emersa Waterbox, LLC",1,,
5,Aug 2 2022 10:05AM,243248,15,"Brookfield Pharmaceuticals, LLC",1,,
6,Aug 2 2022 10:04AM,243174,15,"Brookfield Pharmaceuticals, LLC",37,,
7,Aug 2 2022 10:03AM,243173,15,"Alliance Pharma, Inc.",16,,
8,Aug 2 2022 10:02AM,243247,19,"Emersa Waterbox, LLC",1,,
9,Aug 2 2022 10:00AM,243246,19,"Emersa Waterbox, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 2 2022 10:20AM,243255,10,"Nextsource Biotechnology, LLC",1,,
1,Aug 2 2022 10:19AM,243242,20,"ACI Healthcare USA, Inc.",1,,
2,Aug 2 2022 10:12AM,243252,15,"Carwin Pharmaceutical Associates, LLC",1,,
3,Aug 2 2022 10:08AM,243250,19,"Emersa Waterbox, LLC",1,,
4,Aug 2 2022 10:06AM,243249,19,"Emersa Waterbox, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 2 2022 10:20AM,243255,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
1,Aug 2 2022 10:19AM,243242,20,"ACI Healthcare USA, Inc.",1.0,NaN,NaN
2,Aug 2 2022 10:12AM,243252,15,"Carwin Pharmaceutical Associates, LLC",1.0,NaN,NaN
3,Aug 2 2022 10:08AM,243250,19,"Emersa Waterbox, LLC",1.0,NaN,NaN
4,Aug 2 2022 10:06AM,243249,19,"Emersa Waterbox, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 2 2022 10:20AM,243255,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
1,Aug 2 2022 10:19AM,243242,20,"ACI Healthcare USA, Inc.",1.0,0.0,0.0
2,Aug 2 2022 10:12AM,243252,15,"Carwin Pharmaceutical Associates, LLC",1.0,0.0,0.0
3,Aug 2 2022 10:08AM,243250,19,"Emersa Waterbox, LLC",1.0,0.0,0.0
4,Aug 2 2022 10:06AM,243249,19,"Emersa Waterbox, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3891556,164.0,0.0,0.0
12,486485,4.0,0.0,0.0
15,972847,55.0,0.0,0.0
16,243233,1.0,3.0,0.0
18,243232,1.0,0.0,0.0
19,2432267,26.0,5.0,0.0
20,729639,1.0,39.0,8.0
21,243218,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3891556,164.0,0.0,0.0
1,12,486485,4.0,0.0,0.0
2,15,972847,55.0,0.0,0.0
3,16,243233,1.0,3.0,0.0
4,18,243232,1.0,0.0,0.0
5,19,2432267,26.0,5.0,0.0
6,20,729639,1.0,39.0,8.0
7,21,243218,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,164.0,0.0,0.0
1,12,4.0,0.0,0.0
2,15,55.0,0.0,0.0
3,16,1.0,3.0,0.0
4,18,1.0,0.0,0.0
5,19,26.0,5.0,0.0
6,20,1.0,39.0,8.0
7,21,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,164.0
1,12,Released,4.0
2,15,Released,55.0
3,16,Released,1.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21
Status,,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0
Executing,0.0,0.0,0.0,3.0,0.0,5.0,39.0,1.0
Released,164.0,4.0,55.0,1.0,1.0,26.0,1.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0
1,Executing,0.0,0.0,0.0,3.0,0.0,5.0,39.0,1.0
2,Released,164.0,4.0,55.0,1.0,1.0,26.0,1.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0
1,Executing,0.0,0.0,0.0,3.0,0.0,5.0,39.0,1.0
2,Released,164.0,4.0,55.0,1.0,1.0,26.0,1.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()